In [1]:
  ! pip install wandb

     |████████████████████████████████| 1.8MB 2.9MB/s 
     |████████████████████████████████| 133kB 46.5MB/s 
     |████████████████████████████████| 174kB 33.6MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=02b43e16d7419c5c66ff1610df656e193ec6a1d965ffc4cf55449d5bd1e2ccb8
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=ce823ea8f952447eda46be40b3bb1404b10dda4606c17587b9abea33b59257fe
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [2]:
import wandb

In [3]:
import pandas as pd
import numpy as np
import math
import random
from keras import Input, regularizers, Model
from keras.layers import Input
from keras.layers import LSTM, SimpleRNN, GRU
from keras.layers import Dense, Embedding
from keras.utils.vis_utils import plot_model

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv ("/content/drive/MyDrive/DLassignment3/hi.translit.sampled.train.tsv", sep = '\t')

In [6]:
data.rename(columns = {'अं' : 'Hindi', 'an' : 'English'}, inplace = True)
data

,Hindi,English,3
0,अंकगणित,ankganit,3
1,अंकल,uncle,4
2,अंकुर,ankur,4
3,अंकुरण,ankuran,3
4,अंकुरित,ankurit,3
...,...,...,...
44198,ह्वेनसांग,hiuentsang,1
44199,ह्वेनसांग,hsuantsang,1
44200,ह्वेनसांग,hyensang,1
44201,ह्वेनसांग,xuanzang,1


In [7]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for j in range(len(data)):
  try:
    input_text=data.iloc[j,1]
    target_text=data.iloc[j,0]
    target_text = "\t" + target_text + "\n"
    if(len(input_text)>=1 and len(target_text)>=2):
      input_texts.append(input_text)
      target_texts.append(target_text)
    for char in input_text:
      if char not in input_characters:
        input_characters.add(char)
    for char in target_text:
      if char not in target_characters:
        target_characters.add(char)
  except: print("error")
input_characters.add(' ')
target_characters.add(' ')

error
error


In [8]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 44201
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [9]:
  tmp_x_train = []
  tmp_x_train_index = random.sample([i for i in range(0,np.array(input_texts).shape[0])],math.ceil(np.array(input_texts).shape[0]*0.7))
  tmp_x_val = []
  tmp_y_train = []
  tmp_y_val = []
  for i in tmp_x_train_index:
    tmp_x_train.append(input_texts[i])
    tmp_y_train.append(target_texts[i])
  for i in range(np.array(input_texts).shape[0]):
    if i not in tmp_x_train_index:
      tmp_x_val.append(input_texts[i])
      tmp_y_val.append(target_texts[i])
  for val in tmp_x_train[0]:
    print(input_characters[np.argmax(np.array(val))],end="")
  print()
  for val in tmp_y_train[0]:
    print(target_characters[np.argmax(np.array(val))],end="")
  print()
  for val in tmp_x_val[0]:
    print(input_characters[np.argmax(val)],end="")
  print()
  for val in tmp_y_val[0]:
    print(target_characters[np.argmax(val)],end="")
  print()
  lim=int(len(tmp_x_train)*0.9)
  x_train = tmp_x_train[0:lim]
  y_train = tmp_y_train[0:lim]
  x_val = tmp_x_train[lim:]
  y_val = tmp_y_train[lim:]
  x_test = tmp_x_val
  y_test = tmp_y_val

  # x_train = np.array(sample_in[:math.ceil(np.array(sample_in).shape[0]*0.7)])
  # y_train = np.array(sample_out[:math.ceil(np.array(sample_out).shape[0]*0.7)])
  # x_val = np.array(sample_in[math.ceil(np.array(sample_in).shape[0]*0.7):])
  # y_val = np.array(sample_out[math.ceil(np.array(sample_out).shape[0]*0.7):])
  # x_train = np.array(sample_in)
  # y_train = np.array(sample_out)

In [10]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [11]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [12]:


encoder_input_data = np.zeros(
    (len(x_train), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(x_train), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(x_train), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [13]:
for i, (input_text, target_text) in enumerate(zip(x_train, y_train)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [14]:
encoder_input_val = np.zeros(
    (len(x_val), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_val = np.zeros(
    (len(x_val), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_val = np.zeros(
    (len(x_val), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [15]:
for i, (input_text, target_text) in enumerate(zip(x_val, y_val)):
    for t, char in enumerate(input_text):
        encoder_input_val[i, t, input_token_index[char]] = 1.0
    encoder_input_val[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_val[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_val[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_val[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_val[i, t:, target_token_index[" "]] = 1.0

In [16]:
encoder_input_test = np.zeros(
    (len(x_test), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_test = np.zeros(
    (len(x_test), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_test = np.zeros(
    (len(x_test), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [17]:
for i, (input_text, target_text) in enumerate(zip(x_test, y_test)):
    for t, char in enumerate(input_text):
        encoder_input_test[i, t, input_token_index[char]] = 1.0
    encoder_input_test[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_test[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_test[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_test[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_test[i, t:, target_token_index[" "]] = 1.0

In [18]:
encoder_input_test.shape

(13260, 20, 27)

In [34]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [1,2,3,5,7,9, 10,11,50,60,70,80,90,100]
            #'values':[100,50]
        },
        'weight_decay':{
           'values': [0, 1e-1, 1e-2, 1e-3,0.00099,0.00111]
           # 'values':[1e-3]
          #  'values':[0.00099]
        },
         'batchSize':{
            'values':[64,128]
            # 'values':[8]
            # 'values':[16]
        },
        'regular_dropout':{
            'values':[0.2,0.3,0.4]
            # 'values':[0.2]
            # 'values':[0.4]
        },
        'recurrent_dropout':{
            'values':[0.2,0.3,0.4]
            # 'values':[0.2]
            # 'values':[0.2]
        },
        'regular_activation':{
            'values': ['sigmoid','tanh','relu']
            
            # 'values':['sigmoid']
            # 'values':['relu']
        },
        'recurrent_activation':{
            'values': ['sigmoid','tanh','relu']
            # 'values':['relu']
            # 'values':['sigmoid']
        },
        'no_of_hidden_layers':{
            'values':[1,2,3]
            # 'values':[2]
            # 'values':[3]
        },
        'hidden_layers_size':{
            'values':[64,128,256]
            #'values':[256]
            # 'values':[128]
            # 'values':[16]
        },
        'cell_type':{
            'values':['RNN','GRU','LSTM']
            #'values':['LSTM']
            # 'values':['GRU']
        }
        # 'train_word_accuracy':{
        #     'values':[0]
        # }
        # 'input_embedding_size':{
        #     'values':[16,32,64,256]
        # } 
        
        #'activation': {
            # 'values': ['relu', 'elu', 'selu', 'softmax']
         #   'values': ['sigmoid','tanh','ReLu']
        #}
        
        
    }
}

In [35]:
  sweep_id = wandb.sweep(sweep_config, entity="dl_45_47", project="Transliteration_model")
  #d7f2d3b3afd25dff68841ada514c88e687894723

Create sweep with ID: qdsd7hm1
Sweep URL: https://wandb.ai/dl_45_47/Transliteration_model/sweeps/qdsd7hm1


**REAL**

In [21]:
def RNN_model(config):
  # Define an input sequence and process it.
  encoder_inputs = Input(shape=(None, num_encoder_tokens))
  encoder_inputs_1 = encoder_inputs
  encoder_states_1 = []
  for i in range(config.no_of_hidden_layers-1):
    encoder_inputs_1, state_1 = (SimpleRNN(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(encoder_inputs_1)
    encoder_states_1.append(state_1)
  encoder_inputs_1, state_1 = (SimpleRNN(config.hidden_layers_size, return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(encoder_inputs_1)
  encoder_states_1.append(state_1)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = Input(shape=(None, num_decoder_tokens))
  decoder_inputs_1 = decoder_inputs
  for i in range(config.no_of_hidden_layers):
    decoder_inputs_1, _ = (SimpleRNN(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(decoder_inputs_1, initial_state=encoder_states_1[i])
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', kernel_regularizer=regularizers.l2(config.weight_decay))
  decoder_outputs = (decoder_dense)(decoder_inputs_1)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  # define encoder inference model
  encoder_model = Model(encoder_inputs, encoder_states_1)
  # define decoder inference model
  # Below tensors will hold the states of the previous time step
  decoder_states_inputs = [Input(shape=(config.hidden_layers_size,)) for _ in range(config.no_of_hidden_layers)]

  # To predict the next word in the sequence, set the initial states to the states from the previous time step
  decoder_states = []
  decoder_outputs = decoder_inputs
  for i in range(config.no_of_hidden_layers):
    decoder_outputs, state_1 = (SimpleRNN(config.hidden_layers_size, return_sequences=True, return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(decoder_outputs, initial_state=decoder_states_inputs[i])
    decoder_states.append(state_1)
  
  decoder_outputs_2 = decoder_dense(decoder_outputs)
  decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs_2] + decoder_states)

  return model, encoder_model, decoder_model

In [22]:
def LSTM_model(config):
  # Define an input sequence and process it.
  encoder_inputs = Input(shape=(None, num_encoder_tokens))
  encoder_inputs_1 = encoder_inputs
  encoder_states_1 = []
  for i in range(config.no_of_hidden_layers-1):
    encoder_inputs_1, state_h_1, state_c_1 = (LSTM(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation, recurrent_activation=config.recurrent_activation))(encoder_inputs_1)
    encoder_state = [state_h_1, state_c_1]
    encoder_states_1.append(encoder_state)
  encoder_inputs_1, state_h_1, state_c_1 = (LSTM(config.hidden_layers_size, return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation, recurrent_activation=config.recurrent_activation))(encoder_inputs_1)
  encoder_state = [state_h_1, state_c_1]
  encoder_states_1.append(encoder_state)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = Input(shape=(None, num_decoder_tokens))
  decoder_inputs_1 = decoder_inputs
  for i in range(config.no_of_hidden_layers):
    decoder_inputs_1, _, _ = (LSTM(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation, recurrent_activation=config.recurrent_activation))(decoder_inputs_1, initial_state=encoder_states_1[i])
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', kernel_regularizer=regularizers.l2(config.weight_decay))
  decoder_outputs = decoder_dense(decoder_inputs_1)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  # define encoder inference model
  encoder_model = Model(encoder_inputs, encoder_states_1)
  # define decoder inference model
  # Below tensors will hold the states of the previous time step
  decoder_states_inputs = [Input(shape=(config.hidden_layers_size,)) for _ in range(2*config.no_of_hidden_layers)]

  # To predict the next word in the sequence, set the initial states to the states from the previous time step
  decoder_states = []
  decoder_outputs = decoder_inputs
  for i in range(config.no_of_hidden_layers):
    decoder_outputs, state_h, state_c = (LSTM(config.hidden_layers_size, return_sequences=True, return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation, recurrent_activation=config.recurrent_activation))(decoder_outputs, initial_state=decoder_states_inputs[2*i:2*(i+1)])
    decoder_states.append(state_h)
    decoder_states.append(state_c)
  
  decoder_outputs_2 = decoder_dense(decoder_outputs)
  decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs_2] + decoder_states)

  return model, encoder_model, decoder_model

In [23]:
def GRU_model(config):
  # Define an input sequence and process it.
  encoder_inputs = Input(shape=(None, num_encoder_tokens))
  encoder_inputs_1 = encoder_inputs
  encoder_states_1 = []
  for i in range(config.no_of_hidden_layers-1):
    encoder_inputs_1, state_1 = (GRU(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(encoder_inputs_1)
    encoder_states_1.append(state_1)
  encoder_inputs_1, state_1 = (GRU(config.hidden_layers_size, return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(encoder_inputs_1)
  encoder_states_1.append(state_1)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = Input(shape=(None, num_decoder_tokens))
  decoder_inputs_1 = decoder_inputs
  for i in range(config.no_of_hidden_layers):
    decoder_inputs_1, _ = (GRU(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(decoder_inputs_1, initial_state=encoder_states_1[i])
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', kernel_regularizer=regularizers.l2(config.weight_decay))
  decoder_outputs = decoder_dense(decoder_inputs_1)

   # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  # define encoder inference model
  encoder_model = Model(encoder_inputs, encoder_states_1)
  # define decoder inference model
  # Below tensors will hold the states of the previous time step
  decoder_states_inputs = [Input(shape=(config.hidden_layers_size,)) for _ in range(config.no_of_hidden_layers)]

  # To predict the next word in the sequence, set the initial states to the states from the previous time step
  decoder_states = []
  decoder_outputs = decoder_inputs
  for i in range(config.no_of_hidden_layers):
    decoder_outputs, state_1 = (GRU(config.hidden_layers_size, return_sequences=True, return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(decoder_outputs, initial_state=decoder_states_inputs[i])
    decoder_states.append(state_1)
  
  decoder_outputs_2 = decoder_dense(decoder_outputs)
  decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs_2] + decoder_states)

  return model, encoder_model, decoder_model

In [24]:
def decode_sequence(input_seq,encoder_model,decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        returned = decoder_model.predict([target_seq] + states_value)
        output_tokens = returned[0]
        states_value = returned[1:]
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        
    return decoded_sentence

In [25]:
# def final_gru_rnn_predict_sequence(ienc, idec, source,dest, time_steps, dec_size):
#   # encode
#   state = ienc.predict(source)
 
#   # # # start of sequence input
#   # target_seq = np.array([0.0 for _ in range(dec_size)]).reshape(1, 1, dec_size)
#   target_seq = dest[0][0].reshape(1, 1, dec_size)
#   # collect predictions
#   output = list()
#   for t in range(time_steps-1):
#     # predict next char
#     returned = idec.predict([target_seq] + state)
#     yhat = returned[0]
#     state = returned[1:]
#     # store prediction
#     output.append(yhat[0,0,:])
#     # update target sequence
#     target_seq = dest[0][t+1].reshape(1, 1, dec_size)
#     print(characters[np.argmax(dest[0][t])],end="")
#   return np.array(output)  

In [26]:
# def final_lstm_predict_sequence(ienc, idec, source,dest, time_steps, dec_size):
#   # encode
#   state = ienc.predict(source)
 
#   # # # start of sequence input
#   # target_seq = np.array([0.0 for _ in range(dec_size)]).reshape(1, 1, dec_size)
#   target_seq = dest[0][0].reshape(1, 1, dec_size)
#   # collect predictions
#   output = list()
#   for t in range(time_steps-1):
#     # predict next char
#     returned = idec.predict([target_seq] + state)
#     yhat = returned[0]
#     state = returned[1:]
#     # store prediction
#     output.append(yhat[0,0,:])
#     # update target sequence
#     target_seq = dest[0][t+1].reshape(1, 1, dec_size)
#     print(characters[np.argmax(dest[0][t])],end=" ")
#   return np.array(output)  

In [27]:
tmp_y_val[0][1:-1]

'अंकगणित'

In [28]:
y_train[0][1:-1]

'बडौदा'

In [29]:
y_test[0]

'\tअंकगणित\n'

In [30]:
def accuracy_train( encoder_model, decoder_model):
  count_test=0
  count_train=0
  num_sam=100
  for i in range(num_sam):
    seq_index=random.randint(0,len(x_test))
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_test[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,encoder_model, decoder_model)
    if(decoded_sentence[0:-1]==y_test[seq_index][1:-1]):
      count_test=count_test+1
    print("test_Input word:", x_test[seq_index])
    print("test_expected word:", y_test[seq_index][1:-1])
    print("test_predicted word:", decoded_sentence[0:-1])
    print(" ")
  for i in range(num_sam):
    seq_index=random.randint(0,len(x_train))
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,encoder_model, decoder_model)
    if(decoded_sentence[0:-1]==y_train[seq_index][1:-1]):
      count_train=count_train+1
    print("train_Input word:", x_train[seq_index])
    print("train_expected word:", y_train[seq_index][1:-1])
    print("train_predicted word:", decoded_sentence[0:-1])
    print(" ")
  return count_test/num_sam,count_train/num_sam

In [31]:
# def accuracy_train( encoder_model, decoder_model):
#   cnt=0
#   for i in range(len(tmp_x_val)):
#     # start of sequence input
#     target_seq = np.array([0.0 for _ in range(num_decoder_tokens)]).reshape(1, 1, num_decoder_tokens)

#     if cell_type == 'RNN':
#       pred_val = final_gru_rnn_predict_sequence(encoder_model, decoder_model, np.array([x_train[i]]),np.array([y_train[i]]), time_step, num_decoder_tokens)
#     elif cell_type == 'LSTM':
#       pred_val = final_lstm_predict_sequence(encoder_model, decoder_model, np.array([x_train[i]]),np.array([y_train[i]]), time_step, num_decoder_tokens)
#     elif cell_type == 'GRU':
#       pred_val = final_gru_rnn_predict_sequence(encoder_model, decoder_model, np.array([x_train[i]]),np.array([y_train[i]]), time_step, num_decoder_tokens)
#     print()
#     print("Expected : ",end="\n")
#     for val in y_train[i]:
#       print(characters[np.argmax(val)],end =" ")
#     print()
#     print("Predicted :",end="\n")
#     for val in pred_val:
#       print(characters[np.argmax(val)],end =" ")
#     print()
#   #   cntTemp = 0
#   #   for j in range(max_decoder_seq_length):
#   #     if characters[np.argmax(y_train[i][j])] == characters[np.argmax(pred_val[j])]:
#   #        cntTemp+=1
#   #   if cntTemp == max_decoder_seq_length:
#   #     cnt+=1
#   # return cnt/len(x_val)
#     # print(cnt)


In [32]:
# The sweep calls this function with each set of hyperparameters
def train():
  # Default values for hyper-parameters we're going to sweep over
  #

    config_defaults = {
        'epochs': 5,
        'weight_decay': 1e-2,
         'batchSize':16,
        'regular_dropout':0.2,
        'recurrent_dropout':0.2,
        'no_of_hidden_layers':1,
        'hidden_layers_size':256,
        'cell_type':'LSTM'
    }

    # SUFFLE TRAIN AND VAL INPUT
    # tmp_x_train_index = random.sample([i for i in range(np.array(sample_in).shape[0])],math.ceil(np.array(sample_in).shape[0]*0.7))
    # for i in tmp_x_train_index:
    #   tmp_x_train.append(sample_in[i])
    #   tmp_y_train.append(sample_out[i])
    # for i in range(np.array(sample_in).shape[0]):
    #   if i not in tmp_x_train_index:
    #     tmp_x_val.append(sample_in[i])
    #     tmp_y_val.append(sample_out[i])
    # for val in tmp_x_train[0]:
    #   print(alphabets[np.argmax(np.array(val))],end="")
    # print()
    # for val in tmp_y_train[0]:
    #   print(characters[np.argmax(np.array(val))],end="")
    # print()
    # for val in tmp_x_val[0]:
    #   print(alphabets[np.argmax(val)],end="")
    # print()
    # for val in tmp_y_val[0]:
    #   print(characters[np.argmax(val)],end="")
    # print()
    # x_train = np.array(tmp_x_train)
    # y_train = np.array(tmp_y_train)
    # x_val = np.array(tmp_x_val)
    # y_val = np.array(tmp_y_val)


    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    if config.cell_type == 'RNN':
      model, encoder_model, decoder_model = RNN_model(config)
    elif config.cell_type == 'LSTM':
      model, encoder_model, decoder_model = LSTM_model(config)
    elif config.cell_type == 'GRU':
      model, encoder_model, decoder_model = GRU_model(config)
    
    # plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True)
    # plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True)



    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit([encoder_input_data, decoder_input_data],decoder_target_data, epochs=config.epochs,validation_data=([encoder_input_val, decoder_input_val],decoder_target_val), batch_size=config.batchSize)

    acc_test,acc_train=accuracy_train( encoder_model, decoder_model)    

    accuracy, loss, val_accuracy, val_loss = history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss']

    # wandb.log({'val_loss' : val_loss, 'val_accuracy': val_accuracy})

    for i in range(config.epochs):
      wandb.log({'val_loss' : val_loss[i], 'val_accuracy': val_accuracy[i],'loss' : loss[i], 'accuracy': accuracy[i], 'epoch': i})
    # wandb.finish()
    print("test word accuracy:",acc_test)
    print("train word accuracy:",acc_train)
    
    

In [ ]:
# 18 May
wandb.agent(sweep_id, train)